In [1]:
! nvidia-smi

Wed Jul 16 07:05:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.04              Driver Version: 576.52         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090 Ti     On  |   00000000:01:00.0  On |                  Off |
|  0%   42C    P8             30W /  450W |     307MiB /  24564MiB |      8%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


## CUDA Python 탑재여부 확인 

In [3]:
import torch

print(f"CUDA 사용여부 : {torch.cuda.is_available()}") 
print("사용 가능한 GPU 수:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("현재 사용 중인 GPU:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA를 사용할 수 없습니다.")

CUDA 사용여부 : True
사용 가능한 GPU 수: 1
현재 사용 중인 GPU: NVIDIA GeForce RTX 3090 Ti


In [3]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-12b-it",
    max_seq_length = 1024,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

==((====))==  Unsloth 2025.7.3: Fast Gemma3 patching. Transformers: 4.53.2.
   \\   /|    NVIDIA GeForce RTX 3090 Ti. Num GPUs = 1. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
# Custom CUDA Settings 

from unsloth import FastModel

model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [5]:
# 훈련용 chat template 

from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [6]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForConditionalGeneration(
      (model): Gemma3Model(
        (vision_tower): SiglipVisionModel(
          (vision_model): SiglipVisionTransformer(
            (embeddings): SiglipVisionEmbeddings(
              (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
              (position_embedding): Embedding(4096, 1152)
            )
            (encoder): SiglipEncoder(
              (layers): ModuleList(
                (0-26): 27 x SiglipEncoderLayer(
                  (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                  (self_attn): SiglipAttention(
                    (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                    (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                    (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                    (out_proj): Linear(in_fea

In [7]:
from datasets import load_from_disk
from datasets import Dataset

dataset = load_from_disk("./hf_dataset")

In [8]:
# 허깅페이스 데이터셋용
def transform_to_gemma3(data):
    return {
        "conversations": [
            {
                "from": "human", 
                "value": data['QUESTION']
            },
            {
                "from": "gpt",
                "value": data['ANSWER']
            }
        ]
    }

# 허깅페이스 데이터셋에 적용
transformed_dataset = dataset.map(transform_to_gemma3, remove_columns=['QUESTION', 'ANSWER'])

In [9]:
transformed_dataset

Dataset({
    features: ['conversations'],
    num_rows: 19679
})

In [10]:
from unsloth.chat_templates import standardize_data_formats

datasets = standardize_data_formats(transformed_dataset)

In [11]:
datasets[1]

{'conversations': [{'content': '2023년 농업여건 변화에서 인구감소와 고령화가 차지하는 비율은 각각 얼마인가요?',
   'role': 'user'},
  {'content': '2023년 농업여건 변화에서 인구감소와 고령화는 각각 18.5%와 17.3%를 차지합니다.',
   'role': 'assistant'}]}

In [12]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = datasets.map(formatting_prompts_func, batched = True)

In [13]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [15]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\n수확량 맵핑 시스템에서 사용하는 GPS 기술은 어떤 역할을 하나요?<end_of_turn>\n<start_of_turn>model\nGPS는 수확 위치를 정확히 파악하고, 실시간 수확량 데이터를 지도상에 표시하는 역할을 합니다.<end_of_turn>\n'

In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,679 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 32,735,232 of 12,220,060,272 (0.27% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,4.433500
2,4.034100
3,4.517700
4,4.178600
5,4.053400
6,3.405200
7,3.108600
8,2.916000
9,2.188200
10,2.567700


In [27]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

messages = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": "토경 재배와 수경 재배의 가장 큰 차이점은 무엇인가요?"
    }]
}]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)

result = tokenizer.batch_decode(outputs)
print(result[0])

<bos><start_of_turn>user
토경 재배와 수경 재배의 가장 큰 차이점은 무엇인가요?<end_of_turn>
<start_of_turn>model
토경 재배는 흙이 지지층으로 사용되는 반면, 수경 재배는 식물 뿌리가 수중으로 젖어 있다.<end_of_turn>


In [29]:
if True: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00005.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Unsloth: Merging weights into 16bit:   0%|                                                           | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  20%|██████████                                        | 1/5 [12:45<51:03, 765.96s/it]

...

Unsloth: Merging weights into 16bit:  60%|██████████████████████████████                    | 3/5 [38:50<25:49, 774.58s/it]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  80%|████████████████████████████████████████          | 4/5 [46:09<10:42, 642.09s/it]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████████████████████████████████████████████| 5/5 [52:36<00:00, 631.27s/it]
